# Low-latency item-to-item recommendation system - Creating ANN index

## Overview

This notebook is a part of the series that describes the process of implementing a [**Low-latency item-to-item recommendation system**](https://github.com/jarokaz/analytics-componentized-patterns/tree/master/retail/recommendation-system/bqml-scann).

The notebook demonstrates how to create and deploy an ANN index using item embeddings created in the preceding notebooks. In this notebook you go through the following steps.

1. Exporting embeddings from BigQuery into the JSONL formated file.
2. Creating an ANN Index using the exported embeddings.
3. Creating and ANN Endpoint. 
4. Deploying the ANN Index to the ANN Endpoint.
5. Testing the deployed ANN Index.

This notebook was designed to run on [AI Platform Notebooks](https://cloud.google.com/ai-platform-notebooks). Before running the notebook make sure that you have completed the setup steps as described in the [README file](README.md).


## Setting up the notebook's environment


### Import notebook dependencies

In [1]:
import base64
import datetime
import logging
import os
import json
import pandas as pd
import time
import sys

import grpc

import google.auth
import numpy as np
import tensorflow.io as tf_io

from google.cloud import bigquery
from typing import List, Optional, Text, Tuple

In the experimental release, the *Online Querying API* of the ANN service is exposed throught the GRPC interface. The `ann_grpc` folder contains the grpc client stub to interface to the API.

In [3]:
ANN_GRPC_ENDPOINT_STUB = 'ann_grpc'
if ANN_GRPC_ENDPOINT_STUB not in sys.path:
    sys.path.append(ANN_GRPC_ENDPOINT_STUB)

In [4]:
import ann_grpc.match_pb2_grpc as match_pb2_grpc
import ann_grpc.match_pb2 as match_pb2

### Configure GCP environment

Set the following constants to the values reflecting your environment:

* `PROJECT_ID` - your GCP project ID.
* `PROJECT_NUMBER` - your GCP project number.
* `BQ_DATASET_NAME` - the name of the BigQuery dataset that contains the item embeddings table.
* `BQ_LOCATION` - the dataset location
* `DATA_LOCATION` - a GCS location for the exported embeddings (JSONL) files.
* `VPC_NAME` - a name of the GCP VPC to use for the index deployments. Use the name of the VPC prepared during the initial setup. 
* `REGION` - a compute region. Don't change the default - `us-central` - while the ANN Service is in the experimental stage


In [2]:
PROJECT_ID = 'rec-ai-demo-326116' # Change to your project.
PROJECT_NUMBER = 733956866731
BUCKET = 'rec_bq_jsw' # Change to the bucket you created.
BQ_DATASET_NAME = 'css_retail' # <- CHANGE THIS
BQ_LOCATION = 'US' # <- CHANGE THIS
DATA_LOCATION = f'gs://{BUCKET}/bqml/item_embeddings' # <-CHANGE THIS
VPC_NAME = 'default' # <-CHANGE THIS

EMBEDDINGS_TABLE = 'item_embeddings'
REGION = 'us-central1'
MATCH_SERVICE_PORT = 10000
PEERING_RANGE_NAME='google-reserved-range'

## Setup the VPC Networking if needed

Additional documentation found [here](ann_setup.md)

In [ ]:
!gcloud compute addresses create $PEERING_RANGE_NAME \
  --global \
  --prefix-length=16 \
  --description="peering range for Google service: AI Platform Online Prediction" \
  --network=$VPC_NAME \
  --purpose=VPC_PEERING \
  --project=$PROJECT_ID

In [ ]:
!gcloud services vpc-peerings connect \
  --service=servicenetworking.googleapis.com \
  --network=$VPC_NAME \
  --ranges=$PEERING_RANGE_NAME \
  --project=$PROJECT_ID

## Exporting the embeddings

In the preceeding notebooks you trained the Matrix Factorization BQML model and exported the embeddings to the `item_embeddings` table. 

In this step you will extract the embeddings to a set of JSONL files in the format required by the ANN service.

### Verify the number of embeddings

In [20]:
client = bigquery.Client(project=PROJECT_ID, location=BQ_LOCATION)

In [21]:
query = f"""
    SELECT COUNT(*) embedding_count
    FROM {BQ_DATASET_NAME}.item_embeddings;
"""

query_job = client.query(query)
query_job.to_dataframe()

,embedding_count
0,2933


### Export the embeddings

You will use the [BigQuery export job](https://cloud.google.com/bigquery/docs/exporting-data) to export the embeddings table.

In [22]:
file_name_pattern = 'embedding-*.json'
destination_uri = f'{DATA_LOCATION}/{file_name_pattern}'
table_id = 'item_embeddings'
destination_format = 'NEWLINE_DELIMITED_JSON'

dataset_ref = bigquery.DatasetReference(PROJECT_ID, BQ_DATASET_NAME)
table_ref = dataset_ref.table(table_id)
job_config = bigquery.job.ExtractJobConfig()
job_config.destination_format = bigquery.DestinationFormat.NEWLINE_DELIMITED_JSON

extract_job = client.extract_table(
    table_ref,
    destination_uris=destination_uri,
    job_config=job_config,
    #location=BQ_LOCATION,
)  

extract_job.result()

Inspect the extracted files.

In [23]:
! gsutil ls {DATA_LOCATION}

gs://rec_bq_jsw/bqml/item_embeddings/
gs://rec_bq_jsw/bqml/item_embeddings/embedding-000000000000.json
gs://rec_bq_jsw/bqml/item_embeddings/embeddings-00000-of-00001.csv
gs://rec_bq_jsw/bqml/item_embeddings/tmp/


## Creating an ANN index deployment

Deploying an ANN index is a 3 step process:
1. Creating an index from source files
2. Creating an endpoint to access the index
3. Deploying the index to the endpoint


You will use the REST interface to invoke the AI Platform ANN Service Control Plane API that manages indexes, endpoints, and deployments.

After the index has been deployed you can submit matching requests using Online Querying API. In the experimental stage this API is only accessible through the gRPC interface.


### Define helper classes to encapsulate the ANN Service REST API.

Currently, there is no Python client that encapsulates the ANN Service Control Plane API. The below code snippet defines a simple wrapper that encapsulates a subset of REST APIs used in this notebook.



In [7]:
import datetime
import logging
import json
import time

import google.auth

class ANNClient(object):
    """Base ANN Service client."""
    
    def __init__(self, project_id, project_number, region):
        credentials, _ = google.auth.default()
        self.authed_session = google.auth.transport.requests.AuthorizedSession(credentials)
        self.ann_endpoint = f'{region}-aiplatform.googleapis.com'
        self.ann_parent = f'https://{self.ann_endpoint}/v1alpha1/projects/{project_id}/locations/{region}'
        self.project_id = project_id
        self.project_number = project_number
        self.region = region
        
    def wait_for_completion(self, operation_id, message, sleep_time):
        """Waits for a completion of a long running operation."""
        
        api_url = f'{self.ann_parent}/operations/{operation_id}'

        start_time = datetime.datetime.utcnow()
        while True:
            response = self.authed_session.get(api_url)
            if response.status_code != 200:
                raise RuntimeError(response.json())
            if 'done' in response.json().keys():
                logging.info('Operation completed!')
                break
            elapsed_time = datetime.datetime.utcnow() - start_time
            logging.info('{}. Elapsed time since start: {}.'.format(
                message, str(elapsed_time)))
            time.sleep(sleep_time)
    
        return response.json()


class IndexClient(ANNClient):
    """Encapsulates a subset of control plane APIs 
    that manage ANN indexes."""

    def __init__(self, project_id, project_number, region):
        super().__init__(project_id, project_number, region)

    def create_index(self, display_name, description, metadata):
        """Creates an ANN Index."""
    
        api_url = f'{self.ann_parent}/indexes'
    
        request_body = {
            'display_name': display_name,
            'description': description,
            'metadata': metadata
        }
    
        response = self.authed_session.post(api_url, data=json.dumps(request_body))
        if response.status_code != 200:
            raise RuntimeError(response.text)
        operation_id = response.json()['name'].split('/')[-1]
        
        return operation_id

    def list_indexes(self, display_name=None):
        """Lists all indexes with a given display name or
        all indexes if the display_name is not provided."""
    
        if display_name:
            api_url = f'{self.ann_parent}/indexes?filter=display_name="{display_name}"'
        else:
            api_url = f'{self.ann_parent}/indexes'

        response = self.authed_session.get(api_url).json()

        return response['indexes'] if response else []
    
    def delete_index(self, index_id):
        """Deletes an ANN index."""
        
        api_url = f'{self.ann_parent}/indexes/{index_id}'
        response = self.authed_session.delete(api_url)
        if response.status_code != 200:
            raise RuntimeError(response.text)


class IndexDeploymentClient(ANNClient):
    """Encapsulates a subset of control plane APIs 
    that manage ANN endpoints and deployments."""
    
    def __init__(self, project_id, project_number, region):
        super().__init__(project_id, project_number, region)

    def create_endpoint(self, display_name, vpc_name):
        """Creates an ANN endpoint."""
    
        api_url = f'{self.ann_parent}/indexEndpoints'
        network_name = f'projects/{self.project_number}/global/networks/{vpc_name}'

        request_body = {
            'display_name': display_name,
            'network': network_name
        }

        response = self.authed_session.post(api_url, data=json.dumps(request_body))
        if response.status_code != 200:
            raise RuntimeError(response.text)
        operation_id = response.json()['name'].split('/')[-1]
    
        return operation_id
    
    def list_endpoints(self, display_name=None):
        """Lists all ANN endpoints with a given display name or
        all endpoints in the project if the display_name is not provided."""
        
        if display_name:
            api_url = f'{self.ann_parent}/indexEndpoints?filter=display_name="{display_name}"'
        else:
            api_url = f'{self.ann_parent}/indexEndpoints'

        response = self.authed_session.get(api_url).json()
 
        return response['indexEndpoints'] if response else []
    
    def delete_endpoint(self, endpoint_id):
        """Deletes an ANN endpoint."""
        
        api_url = f'{self.ann_parent}/indexEndpoints/{endpoint_id}'
        
        response = self.authed_session.delete(api_url)
        if response.status_code != 200:
            raise RuntimeError(response.text)
        
        return response.json()
    
    def create_deployment(self, display_name, deployment_id, endpoint_id, index_id):
        """Deploys an ANN index to an endpoint."""
    
        api_url = f'{self.ann_parent}/indexEndpoints/{endpoint_id}:deployIndex'
        index_name = f'projects/{self.project_number}/locations/{self.region}/indexes/{index_id}'

        request_body = {
            'deployed_index': {
                'id': deployment_id,
                'index': index_name,
                'display_name': display_name
            }
        }

        response = self.authed_session.post(api_url, data=json.dumps(request_body))
        if response.status_code != 200:
            raise RuntimeError(response.text)
        operation_id = response.json()['name'].split('/')[-1]
        
        return operation_id
    
    def get_deployment_grpc_ip(self, endpoint_id, deployment_id):
        """Returns a private IP address for a gRPC interface to 
        an Index deployment."""
  
        api_url = f'{self.ann_parent}/indexEndpoints/{endpoint_id}'

        response = self.authed_session.get(api_url)
        if response.status_code != 200:
            raise RuntimeError(response.text)
            
        endpoint_ip = None
        if 'deployedIndexes' in response.json().keys():
            for deployment in response.json()['deployedIndexes']:
                if deployment['id'] == deployment_id:
                    endpoint_ip = deployment['privateEndpoints']['matchGrpcAddress']
                    
        return endpoint_ip

    
    def delete_deployment(self, endpoint_id, deployment_id):
        """Undeployes an index from an endpoint."""
        
        api_url = f'{self.ann_parent}/indexEndpoints/{endpoint_id}:undeployIndex'
        
        request_body = {
            'deployed_index_id': deployment_id
        }
    
        response = self.authed_session.post(api_url, data=json.dumps(request_body))
        if response.status_code != 200:
            raise RuntimeError(response.text)
        
        return response
    
index_client = IndexClient(PROJECT_ID, PROJECT_NUMBER, REGION)
deployment_client = IndexDeploymentClient(PROJECT_ID, PROJECT_NUMBER, REGION)

### Create an ANN index

#### List all indexes registered with the ANN service

In [25]:
indexes = index_client.list_indexes()

if not indexes:
    print('There are not any indexes registered with the service')

for index in indexes:
    print(index['name'])

There are not any indexes registered with the service


#### Configure and create a new index based on the exported embeddings

Index creation is a long running operation. Be patient.

In [26]:
index_display_name = 'Product embeddings'
index_description = 'Product embeddings created BQML Matrix Factorization model'

index_metadata = {
    'contents_delta_uri': DATA_LOCATION,
    'config': {
        'dimensions': 50,
        'approximate_neighbors_count': 50,
        'distance_measure_type': 'DOT_PRODUCT_DISTANCE',
        'feature_norm_type': 'UNIT_L2_NORM',
        'tree_ah_config': {
            'child_node_count': 1000,
            'max_leaves_to_search': 100
         }
    }
}

### Look inside the contents of the `$DATA_LOCATION` bucket - ensure there are no subdirectories and there are either `.json` or `.csv` files (not mixed)

In [29]:
logging.getLogger().setLevel(logging.INFO)

operation_id = index_client.create_index(index_display_name, 
                                          index_description,
                                          index_metadata)

response = index_client.wait_for_completion(operation_id, 'Creating index', 20)
print(response)

INFO:root:Creating index. Elapsed time since start: 0:00:00.078623.
INFO:root:Creating index. Elapsed time since start: 0:00:20.164982.
INFO:root:Creating index. Elapsed time since start: 0:00:40.278203.
INFO:root:Creating index. Elapsed time since start: 0:01:00.414148.
INFO:root:Creating index. Elapsed time since start: 0:01:20.514342.
INFO:root:Creating index. Elapsed time since start: 0:01:40.581232.
INFO:root:Creating index. Elapsed time since start: 0:02:00.652941.
INFO:root:Creating index. Elapsed time since start: 0:02:20.762145.
INFO:root:Creating index. Elapsed time since start: 0:02:40.847362.
INFO:root:Creating index. Elapsed time since start: 0:03:00.918899.
INFO:root:Creating index. Elapsed time since start: 0:03:20.980803.
INFO:root:Creating index. Elapsed time since start: 0:03:41.124582.
INFO:root:Creating index. Elapsed time since start: 0:04:01.208450.
INFO:root:Creating index. Elapsed time since start: 0:04:21.275295.
INFO:root:Creating index. Elapsed time since sta

{'@type': 'type.googleapis.com/google.cloud.aiplatform.v1alpha1.Index', 'name': 'projects/733956866731/locations/us-central1/indexes/3018854309593874432'}


#### Verify that the index was created

In [30]:
indexes = index_client.list_indexes(index_display_name)

for index in indexes:
    print(index['name'])

if indexes: 
    index_id = index['name'].split('/')[-1]
    print(f'Index: {index_id} will be used for deployment')
else:
    print('No indexes available for deployment')

projects/733956866731/locations/us-central1/indexes/3018854309593874432
Index: 3018854309593874432 will be used for deployment


### Create the index deployment

#### List all endpoints registered with the ANN service

In [34]:
endpoints = deployment_client.list_endpoints()

if not endpoints:
    print('There are not any endpoints registered with the service')

for endpoint in endpoints:
    print(endpoint['name'])

There are not any endpoints registered with the service


#### Create an index endpoint

In [10]:
deployment_display_name = 'Products embeddings endpoint'

In [36]:
operation_id = deployment_client.create_endpoint(deployment_display_name, VPC_NAME)

response = index_client.wait_for_completion(operation_id, 'Waiting for endpoint', 10)
print(response)

INFO:root:Waiting for endpoint. Elapsed time since start: 0:00:00.098077.
INFO:root:Operation completed!


{'@type': 'type.googleapis.com/google.cloud.aiplatform.v1alpha1.IndexEndpoint', 'name': 'projects/733956866731/locations/us-central1/indexEndpoints/7573400907748999168'}


#### Verify that the endpoint was created

In [11]:
endpoints = deployment_client.list_endpoints(deployment_display_name)

for endpoint in endpoints:
    print(endpoint['name'])
    
if endpoints: 
    endpoint_id = endpoint['name'].split('/')[-1]
    print(f'Endpoint: {endpoint_id} will be used for deployment')
else:
    print('No endpoints available for deployment')


projects/733956866731/locations/us-central1/indexEndpoints/7573400907748999168
Endpoint: 7573400907748999168 will be used for deployment


#### Deploy the index to the endpoint

##### Set the deployed index ID

The ID of the deployed index must be unique within your project.

In [5]:
deployed_index_id = 'product_embeddings_deployed_ind'

##### Deploy the index

Be patient. Index deployment is a long running operation

In [67]:
response = index_client.wait_for_completion(operation_id, 'Waiting for deployment', 10)

INFO:root:Operation completed!


In [69]:
operation_id = deployment_client.create_deployment(deployment_display_name, 
                                                   deployed_index_id,
                                                   endpoint_id,
                                                   index_id)

response = index_client.wait_for_completion(operation_id, 'Waiting for deployment', 10)
print(response)

INFO:root:Waiting for deployment. Elapsed time since start: 0:00:00.144543.
INFO:root:Waiting for deployment. Elapsed time since start: 0:00:10.199308.
INFO:root:Waiting for deployment. Elapsed time since start: 0:00:20.267708.
INFO:root:Waiting for deployment. Elapsed time since start: 0:00:30.322752.
INFO:root:Waiting for deployment. Elapsed time since start: 0:00:40.375460.
INFO:root:Waiting for deployment. Elapsed time since start: 0:00:50.422122.
INFO:root:Waiting for deployment. Elapsed time since start: 0:01:00.999675.
INFO:root:Waiting for deployment. Elapsed time since start: 0:01:11.065783.
INFO:root:Waiting for deployment. Elapsed time since start: 0:01:21.131241.
INFO:root:Waiting for deployment. Elapsed time since start: 0:01:31.180398.
INFO:root:Waiting for deployment. Elapsed time since start: 0:01:41.239141.
INFO:root:Waiting for deployment. Elapsed time since start: 0:01:51.287332.
INFO:root:Waiting for deployment. Elapsed time since start: 0:02:01.343314.
INFO:root:Wa

{'name': 'projects/733956866731/locations/us-central1/indexEndpoints/7573400907748999168/operations/7301028512926793728', 'metadata': {'@type': 'type.googleapis.com/google.cloud.aiplatform.v1alpha1.DeployIndexOperationMetadata', 'genericMetadata': {'createTime': '2021-09-20T22:31:55.542198Z', 'updateTime': '2021-09-20T22:44:33.355559Z'}, 'deployedIndexId': 'product_embeddings_deployed_ind'}, 'done': True, 'response': {'@type': 'type.googleapis.com/google.cloud.aiplatform.v1alpha1.DeployIndexResponse', 'deployedIndex': {'id': 'product_embeddings_deployed_ind'}}}


## Querying the ANN service

You will use the gRPC interface to query the deployed index.

### Retrieve the gRPC private endpoint for the ANN Match service

In [12]:
deployed_index_ip = deployment_client.get_deployment_grpc_ip(endpoint_id, deployed_index_id)
endpoint = f'{deployed_index_ip}:{MATCH_SERVICE_PORT}'
print(f'gRPC endpoint for the: {deployed_index_id} deployment is: {endpoint}')

gRPC endpoint for the: product_embeddings_deployed_ind deployment is: 10.120.0.5:10000


### Create a helper wrapper around the Match Service gRPC API.

The wrapper uses the pre-generated gRPC stub to the Online Querying gRPC interface. 

In [13]:
class MatchService(object):
    """This is a wrapper around Online Querying gRPC interface."""
    def __init__(self, endpoint, deployed_index_id):
        self.endpoint = endpoint
        self.deployed_index_id = deployed_index_id

    def single_match(
        self,
        embedding: List[float],
        num_neighbors: int) -> List[Tuple[str, float]]:
        """Requests a match for a single embedding."""
    
        match_request = match_pb2.MatchRequest(deployed_index_id=self.deployed_index_id,
                                               float_val=embedding,
                                               num_neighbors=num_neighbors)
        with grpc.insecure_channel(endpoint, options=(('grpc.enable_http_proxy', 0),)) as channel:
            stub = match_pb2_grpc.MatchServiceStub(channel)
            response = stub.Match(match_request)
    
        return [(neighbor.id, neighbor.distance) for neighbor in response.neighbor]


    def batch_match(
        self,
        embeddings: List[List[float]],
        num_neighbors: int) -> List[List[Tuple[str, float]]]:
        """Requests matches ofr a list of embeddings."""
    
        match_requests = [
            match_pb2.MatchRequest(deployed_index_id=self.deployed_index_id,
                                   float_val=embedding,
                                   num_neighbors=num_neighbors)
            for embedding in embeddings]
    
        batches_per_index = [
            match_pb2.BatchMatchRequest.BatchMatchRequestPerIndex(
                deployed_index_id=self.deployed_index_id,
                requests=match_requests)]
    
        batch_match_request = match_pb2.BatchMatchRequest(
            requests=batches_per_index)
    
        with grpc.insecure_channel(endpoint) as channel:
            stub = match_pb2_grpc.MatchServiceStub(channel)
            response = stub.BatchMatch(batch_match_request)
        
        matches = []
        for batch_per_index in response.responses:
            for match in batch_per_index.responses:
                matches.append(
                    [(neighbor.id, neighbor.distance) for neighbor in match.neighbor])
        
        return matches
    
match_service = MatchService(endpoint, deployed_index_id)

### Prepare sample data

Retrieve a few embeddings from the BigQuery embedding table

In [112]:
from ann_grpc.match_pb2 import MatchRequest,  MatchResponse, BatchMatchRequest, BatchMatchResponse
channel = grpc.insecure_channel(endpoint, options=(('grpc.enable_http_proxy', 0),))


mat = channel.unary_unary(
        '/google.cloud.aiplatform.container.v1alpha1.MatchService/Match',
        request_serializer=MatchRequest.SerializeToString,
        response_deserializer=MatchResponse.FromString,
        )
bat_mat = channel.unary_unary(
        '/google.cloud.aiplatform.container.v1alpha1.MatchService/BatchMatch',
        request_serializer=BatchMatchRequest.SerializeToString,
        response_deserializer=BatchMatchResponse.FromString,
        )

In [20]:
!unset http_proxy; unset https_proxy
!unset http_proxy; unset https_proxy

In [21]:
match_request = match_pb2.MatchRequest(deployed_index_id=deployed_index_id,
                                               float_val=sample_embeddings[0],
                                               num_neighbors=1)

with grpc.insecure_channel(endpoint) as channel:
    stub = match_pb2_grpc.MatchServiceStub(channel)
    response = stub.Match(match_request)

_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNIMPLEMENTED
	details = ""
	debug_error_string = "{"created":"@1632180762.586950560","description":"Error received from peer ipv4:10.120.0.5:10000","file":"src/core/lib/surface/call.cc","file_line":1069,"grpc_message":"","grpc_status":12}"
>

In [15]:
%%bigquery df_embeddings

SELECT item_id, embedding
FROM `rec-ai-demo-326116.css_retail.item_embeddings`
LIMIT 100

Downloading: 100%|██████████| 100/100 [00:00<00:00, 104.95rows/s]


In [16]:
sample_embeddings = [list(embedding) for embedding in df_embeddings['embedding']]
sample_embeddings[0]

[-3.806297101245312,
 -15.78938341682279,
 10.878980769866642,
 5.760965677607754,
 -2.7687092376605698,
 -12.171721530121607,
 2.2283898258915977,
 -5.271758604215919,
 -24.498092482700436,
 4.39482614207085,
 -2.0724808184591166,
 33.44620651536166,
 -7.893344359345121,
 6.551653064580407,
 -2.1018886213217414,
 29.703981647415986,
 -10.146021658133876,
 0.2748278059830755,
 11.25628636636933,
 23.72181121481588,
 19.976807113372054,
 9.759315510500064,
 3.651946215848869,
 8.56964901856156,
 -18.746348404618665,
 2.887023670920412,
 -10.407702836744022,
 2.1968258871235267,
 -3.363959404070823,
 2.5375225633190035,
 -3.5019245211637315,
 -3.8072199642980755,
 -0.9097114055869673,
 5.226877233859579,
 -8.936797602271083,
 11.98517325507575,
 13.745920006956775,
 5.04213765955379,
 6.46930988155931,
 1.9311396935945007,
 -4.130503033980623,
 -14.32082046954076,
 4.013387978737336,
 -20.364025222230776,
 -5.635485662102075,
 -1.220635864599934,
 -0.14946465752155927,
 6.127294895340738

### Run a single match query

The following call requests 10 closest neighbours for a single embedding.

In [17]:
%%time 

single_match = match_service.single_match(sample_embeddings[50], 10)
single_match

_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNIMPLEMENTED
	details = ""
	debug_error_string = "{"created":"@1632180730.156216715","description":"Error received from peer ipv4:10.120.0.5:10000","file":"src/core/lib/surface/call.cc","file_line":1069,"grpc_message":"","grpc_status":12}"
>

### Run a batch match query

The following call requests 3 closest neighbours for each of the embeddings in a batch of 5.

In [53]:
%%time 

batch_match = match_service.batch_match(sample_embeddings[0:5], 3)
batch_match

_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "DNS resolution failed for service: None:10000"
	debug_error_string = "{"created":"@1632176279.746656666","description":"Resolver transient failure","file":"src/core/ext/filters/client_channel/client_channel.cc","file_line":1357,"referenced_errors":[{"created":"@1632176279.746653978","description":"DNS resolution failed for service: None:10000","file":"src/core/ext/filters/client_channel/resolver/dns/c_ares/dns_resolver_ares.cc","file_line":359,"grpc_status":14,"referenced_errors":[{"created":"@1632176279.746614606","description":"C-ares status is not ARES_SUCCESS qtype=A name=None is_balancer=0: Domain name not found","file":"src/core/ext/filters/client_channel/resolver/dns/c_ares/grpc_ares_wrapper.cc","file_line":724,"referenced_errors":[{"created":"@1632176279.725097981","description":"C-ares status is not ARES_SUCCESS qtype=AAAA name=None is_balancer=0: Domain name not fou

## Clean up

**WARNING**

The below code will delete all ANN deployments, endpoints, and indexes in the configured project.

### Delete index deployments and endpoints

In [11]:
for endpoint in deployment_client.list_endpoints():
    endpoint_id = endpoint['name'].split('/')[-1]
    if 'deployedIndexes' in endpoint.keys():
        for deployment in endpoint['deployedIndexes']:
            print('   Deleting index deployment: {} in the endpoint: {} '.format(deployment['id'], endpoint_id))
            deployment_client.delete_deployment(endpoint_id, deployment['id'])
    print('Deleting endpoint: {}'.format(endpoint['name']))
    deployment_client.delete_endpoint(endpoint_id)

Deleting endpoint: projects/895222332033/locations/us-central1/indexEndpoints/856246879153815552
Deleting endpoint: projects/895222332033/locations/us-central1/indexEndpoints/3621457050359300096
Deleting endpoint: projects/895222332033/locations/us-central1/indexEndpoints/108649341010313216


### Delete indexes

In [12]:
for index in index_client.list_indexes():
    index_id = index['name'].split('/')[-1]
    print('Deleting index: {}'.format(index['name']))
    index_client.delete_index(index_id)

Deleting index: projects/895222332033/locations/us-central1/indexes/1335880239468773376
Deleting index: projects/895222332033/locations/us-central1/indexes/3544895856694001664
Deleting index: projects/895222332033/locations/us-central1/indexes/4886968545650409472


## License

Copyright 2020 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License. You may obtain a copy of the License at: http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. 

See the License for the specific language governing permissions and limitations under the License.

**This is not an official Google product but sample code provided for an educational purpose**